In [1]:
import sys
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", module="IPython")

root_dir = Path().absolute()
# Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
if root_dir.parts[-1:] == ('notebooks',):
    root_dir = Path(*root_dir.parts[:-1])
    sys.path.append(str(root_dir))
if root_dir.parts[-1:] == ('ccfraud',):
    root_dir = Path(*root_dir.parts[:-1])
    sys.path.append(str(root_dir))
root_dir = str(root_dir) 

print(f"Root dir: {root_dir}")

# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Root dir: /home/jdowling/Projects/mlfs-book
HopsworksSettings initialized!


In [2]:
import hopsworks

proj = hopsworks.login()
fs = proj.get_feature_store()
mr = proj.get_model_registry()

2025-12-29 12:03:05,209 INFO: Initializing external client
2025-12-29 12:03:05,211 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2025-12-29 12:03:05,936 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/120


In [3]:
merchant_fg = fs.get_feature_group("merchant_details", version=1)
account_fg = fs.get_feature_group("account_details", version=1)
bank_fg = fs.get_feature_group("bank_details", version=1)
card_fg = fs.get_feature_group("card_details", version=1)
cc_trans_aggs_fg = fs.get_feature_group("cc_trans_aggs_fg", version=1)
cc_trans_fg = fs.get_feature_group("cc_trans_fg", version=1)

In [14]:
dff = cc_trans_fg.read()
dff

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.23s) 


,t_id,cc_num,merchant_id,account_id,amount,time_since_last_trans,days_to_card_expiry,is_fraud,ts,haversine_distance
0,0,3078-3174-5090-3396,48,707,171.30,8567,0,False,2025-09-07 16:15:38,False
1,1,3078-3174-5090-3396,6,707,149.26,33220,0,False,2025-09-28 05:14:54,True
2,2,3078-3174-5090-3396,33,707,11.88,332016,0,False,2025-09-24 15:17:29,True
3,3,3078-3174-5090-3396,33,707,14.97,15197,0,False,2025-09-18 03:59:32,True
4,4,3078-3174-5090-3396,1,707,15.18,161267,0,False,2025-09-17 23:46:15,True
...,...,...,...,...,...,...,...,...,...,...
50044,50044,1717-3126-3926-1512,20,752,25.49,8,0,True,2025-09-21 12:36:41,True
50045,50045,7292-9383-3619-7204,49,914,60.10,43686,0,True,2025-09-16 14:58:57,True
50046,50046,7292-9383-3619-7204,13,914,28.05,359,0,True,2025-09-16 15:04:56,False
50047,50047,3234-7406-9941-5832,37,236,72.04,66550,0,True,2025-09-08 19:45:33,True


In [15]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50049 entries, 0 to 50048
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   t_id                   50049 non-null  int64         
 1   cc_num                 50049 non-null  object        
 2   merchant_id            50049 non-null  int64         
 3   account_id             50049 non-null  int64         
 4   amount                 50049 non-null  float64       
 5   time_since_last_trans  50049 non-null  int64         
 6   days_to_card_expiry    50049 non-null  int64         
 7   is_fraud               50049 non-null  bool          
 8   ts                     50049 non-null  datetime64[us]
 9   haversine_distance     50049 non-null  bool          
dtypes: bool(2), datetime64[us](1), float64(1), int64(5), object(1)
memory usage: 3.2+ MB


In [10]:
dfa = cc_trans_aggs_fg.read()
dfa

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.57s) 


,cc_num,account_id,bank_id,num_trans_last_10_mins,sum_trans_last_10_mins,num_trans_last_hour,sum_trans_last_hour,num_trans_last_day,sum_trans_last_day,num_trans_last_week,sum_trans_last_week,prev_card_present,prev_ip_transaction,prev_ts_transaction,event_time
0,1003-1567-5660-1267,914.0,20.0,1,24.75,1,24.75,1,24.75,1,24.75,None,37.94.0.177,NaT,2025-09-05 18:25:41
1,1003-1567-5660-1267,914.0,20.0,1,25.84,1,25.84,1,25.84,1,25.84,None,2.90.12.104,NaT,2025-09-08 21:56:11
2,1003-1567-5660-1267,914.0,20.0,1,227.51,1,227.51,2,253.35,2,253.35,False,5.8.219.248,2025-09-08 21:56:11,2025-09-09 01:53:52
3,1003-1567-5660-1267,914.0,20.0,1,40.05,1,40.05,1,40.05,3,293.40,False,None,2025-09-09 01:53:52,2025-09-14 03:18:33
4,1003-1567-5660-1267,914.0,20.0,1,76.81,1,76.81,2,116.86,4,370.21,True,85.107.169.159,2025-09-14 03:18:33,2025-09-14 06:46:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50043,9996-1002-2489-5990,165.0,45.0,1,149.85,1,149.85,1,149.85,5,877.81,False,None,2025-09-27 11:48:18,2025-09-29 19:14:05
50044,9996-1002-2489-5990,165.0,45.0,1,5.09,1,5.09,2,154.94,6,882.90,False,46.241.231.251,2025-09-29 19:14:05,2025-09-30 03:35:34
50045,9996-1002-2489-5990,165.0,45.0,1,420.13,1,420.13,1,420.13,7,1303.03,False,46.22.197.141,2025-09-30 03:35:34,2025-10-02 02:46:07
50046,9996-1002-2489-5990,165.0,45.0,1,15.16,1,15.16,2,435.29,8,1318.19,True,78.227.138.207,2025-10-02 02:46:07,2025-10-02 06:31:00


In [11]:
dfa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50048 entries, 0 to 50047
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   cc_num                  50048 non-null  object        
 1   account_id              48693 non-null  float64       
 2   bank_id                 48693 non-null  float64       
 3   num_trans_last_10_mins  50048 non-null  int64         
 4   sum_trans_last_10_mins  50048 non-null  float64       
 5   num_trans_last_hour     50048 non-null  int64         
 6   sum_trans_last_hour     50048 non-null  float64       
 7   num_trans_last_day      50048 non-null  int64         
 8   sum_trans_last_day      50048 non-null  float64       
 9   num_trans_last_week     50048 non-null  int64         
 10  sum_trans_last_week     50048 non-null  float64       
 11  prev_card_present       46873 non-null  object        
 12  prev_ip_transaction     46951 non-null  object

In [8]:
df2 = account_fg.read()
df2

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.52s) 


,account_id,name,address,debt_end_prev_month,last_modified,creation_date,end_date,home_country
0,913,Emily Nelson,"USCGC Stone, FPO AA 73728",4942.59,2025-03-24 00:00:00+00:00,2025-05-21 00:00:00+00:00,NaT,Poland
1,252,Tanya Kim,"3275 Mccormick Harbor, Port Kimberlyfort, NJ 9...",282.12,2025-03-22 00:00:00+00:00,2020-12-26 00:00:00+00:00,NaT,France
2,708,Joseph Williams,"54384 Tracy Expressway Apt. 344, Ferrellmouth,...",1775.22,2024-11-01 00:00:00+00:00,2024-12-14 00:00:00+00:00,NaT,France
3,640,Michael Stanley,"09345 Morrow Manor Apt. 282, West Natasha, DE ...",1485.41,2025-06-11 00:00:00+00:00,2023-04-15 00:00:00+00:00,NaT,Canada
4,941,Virginia Casey,"PSC 7339, Box 3049, APO AE 57051",1144.14,2024-11-07 00:00:00+00:00,2021-03-14 00:00:00+00:00,NaT,Spain
...,...,...,...,...,...,...,...,...
995,520,Amy Martinez,"74584 Alison Grove Suite 881, West Rachelfort,...",2933.75,2024-10-20 00:00:00+00:00,2025-02-23 00:00:00+00:00,2028-06-03 00:00:00+00:00,Spain
996,837,Carol Sanders,"6427 Samuel Dale Apt. 376, New William, FL 14895",31.90,2025-07-30 00:00:00+00:00,2023-12-11 00:00:00+00:00,NaT,Poland
997,531,Mark Jimenez,"8895 Nicholas Prairie, Lake Jessicafurt, WV 85594",3207.12,2025-01-19 00:00:00+00:00,2023-08-19 00:00:00+00:00,NaT,Canada
998,866,Lori Mason,"3888 Maria Harbor, Jessicaberg, NH 80467",1597.95,2025-03-07 00:00:00+00:00,2024-03-10 00:00:00+00:00,NaT,Spain


In [9]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype                  
---  ------               --------------  -----                  
 0   account_id           1000 non-null   int64                  
 1   name                 1000 non-null   object                 
 2   address              1000 non-null   object                 
 3   debt_end_prev_month  1000 non-null   float64                
 4   last_modified        1000 non-null   datetime64[us, Etc/UTC]
 5   creation_date        1000 non-null   datetime64[us, Etc/UTC]
 6   end_date             99 non-null     datetime64[us, Etc/UTC]
 7   home_country         1000 non-null   object                 
dtypes: datetime64[us, Etc/UTC](3), float64(1), int64(1), object(3)
memory usage: 62.6+ KB


In [12]:
# 'account_id', 'bank_id', 
#    .join(account_fg.select_all(), on="account_id")\
#cc_trans_aggs_fg.select_except(['prev_ip_transaction', 'prev_ts_transaction', 'event_time'])\
#     .join(bank_fg.select(['credit_rating', 'days_since_bank_cr_changed', 'country']), on="bank_id")
subtree1 = cc_trans_aggs_fg.select_all()\
    .join(account_fg.select_all(), on="account_id")

In [13]:
subtree1.show(10)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.40s) 


,cc_num,account_id,bank_id,num_trans_last_10_mins,sum_trans_last_10_mins,num_trans_last_hour,sum_trans_last_hour,num_trans_last_day,sum_trans_last_day,num_trans_last_week,...,prev_ts_transaction,event_time,account_details_account_id,account_details_name,account_details_address,account_details_debt_end_prev_month,account_details_last_modified,account_details_creation_date,account_details_end_date,account_details_home_country
0,1352-3884-5289-2657,NaN,NaN,1,83.81,1,83.81,1,83.81,1,...,NaT,2025-09-05 09:05:43,NaN,None,None,NaN,NaT,NaT,NaT,None
1,1352-3884-5289-2657,NaN,NaN,1,28.23,1,28.23,2,112.04,2,...,2025-09-05 09:05:43,2025-09-05 11:47:50,NaN,None,None,NaN,NaT,NaT,NaT,None
2,1352-3884-5289-2657,NaN,NaN,1,70.98,1,70.98,1,70.98,1,...,NaT,2025-09-05 12:19:55,NaN,None,None,NaN,NaT,NaT,NaT,None
3,1352-3884-5289-2657,NaN,NaN,1,5.41,1,5.41,4,188.43,4,...,2025-09-05 12:19:55,2025-09-06 03:29:46,NaN,None,None,NaN,NaT,NaT,NaT,None
4,1352-3884-5289-2657,NaN,NaN,1,70.75,1,70.75,1,70.75,4,...,2025-09-05 12:19:55,2025-09-06 17:00:41,NaN,None,None,NaN,NaT,NaT,NaT,None
5,1352-3884-5289-2657,NaN,NaN,1,40.47,1,40.47,2,111.22,5,...,2025-09-06 17:00:41,2025-09-06 18:58:51,NaN,None,None,NaN,NaT,NaT,NaT,None
6,1352-3884-5289-2657,NaN,NaN,1,38.76,1,38.76,1,38.76,7,...,2025-09-06 18:58:51,2025-09-11 11:14:36,NaN,None,None,NaN,NaT,NaT,NaT,None
7,1352-3884-5289-2657,NaN,NaN,1,16.93,1,16.93,2,55.69,8,...,2025-09-11 11:14:36,2025-09-11 21:52:58,NaN,None,None,NaN,NaT,NaT,NaT,None
8,1352-3884-5289-2657,NaN,NaN,1,34.88,1,34.88,1,34.88,3,...,2025-09-06 18:58:51,2025-09-12 20:01:29,NaN,None,None,NaN,NaT,NaT,NaT,None
9,1352-3884-5289-2657,NaN,NaN,1,89.33,1,89.33,2,124.21,4,...,2025-09-12 20:01:29,2025-09-13 13:49:02,NaN,None,None,NaN,NaT,NaT,NaT,None


In [ ]:
selection = cc_trans_fg.select_except(['t_id', 'cc_num', 'merchant_id', 'account_id', 'ip_address', 'ts'])\
    .join(merchant_fg.select_features(), prefix="merchant_", on="merchant_id")\
    .join(subtree1, on="cc_num")

In [ ]:
df = selection.show(100)
df

In [ ]:
fv = fs.get_or_create_feature_view(name="cc_fraud_fv", 
                                   version=1, 
                                   description="features for a credit card fraud prediction model",
                                   query=selection,
                                   labels=['is_fraud']
                                  )

In [ ]:
test_start="2025-09-15 00:00"
X_train, X_test, y_train, y_test = fv.train_test_split(test_start=test_start)
X_train